In [ ]:
from pathlib import Path
import subprocess
import os
from glob import glob
import numpy as np
import pandas as pd
from datetime import datetime
import joblib
import warnings
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.metrics import roc_auc_score
from tqdm.auto import tqdm
from sklearn.model_selection import TimeSeriesSplit, GroupKFold, StratifiedGroupKFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.datasets import load_wine
import pickle
import seaborn as sns

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.load("nllb-200-3.3B/nllb_model.pt", weights_only = False)
model.to(device)
model.eval()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nllb-200-3.3B/tokenize")


In [ ]:
import torch
print(torch.__version__)           # Should show 2.2.2+cu118
print(torch.cuda.is_available())   # Should return True

In [ ]:
data_train = pd.read_json(path_or_buf="Data_MT/mt_train.jsonl", lines=True)
data_val = pd.read_json(path_or_buf="Data_MT/mt_dev.jsonl", lines=True)
data_test = pd.read_json(path_or_buf="Data_MT/mt_test.jsonl", lines=True)

In [ ]:
data_train = data_train.drop('context',axis=1)
data_val = data_val.drop('context',axis=1)
data_test = data_test.drop('context',axis=1)

In [ ]:
data_train

In [ ]:
from datasets import Dataset, DatasetDict
from transformers import (
    NllbTokenizer, 
    M2M100ForConditionalGeneration,
    Trainer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
from torch.utils.data import DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
import logging
from IPython.display import display, HTML
import matplotlib.pyplot as plt

class Config:
    SRC_LANG = "zho_Hans"  # Simplified Chinese
    TGT_LANG = "tha_Thai"  # Thai
    
    # Training parameters
    OUTPUT_DIR = "./chinese-thai-translation-model"
    NUM_EPOCHS = 12
    BATCH_SIZE = 2  # Adjust based on your GPU memory
    LEARNING_RATE = 1e-4
    MAX_LENGTH = 1024

config = Config()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    data_val['source'], data_val['translation'], 
    test_size=0.1,  # 20% for test set
    random_state=42,  # for reproducibility
    shuffle=True,
)

In [ ]:
result = pd.concat([data_train, pd.DataFrame({"source": x_train, "translation": y_train})], axis=0)
# result = pd.concat([data_train, data_val, axis=0)


In [ ]:
datasets = DatasetDict({
    'train': Dataset.from_dict({'source': result['source'], 'translation': result['translation']}),
    'validation': Dataset.from_dict({'source': x_test, 'translation': y_test}),
})

In [ ]:
datasets

In [ ]:
tokenizer.src_lang = config.SRC_LANG
tokenizer.tgt_lang = config.TGT_LANG

In [ ]:
# Tokenize with context (Option 2 - Batched)
def tokenize_function(examples):
    inputs = tokenizer(
        examples["source"],
        truncation=True,
        max_length=1024,
        return_tensors="pt",
        # return_overflowing_tokens=True,  # Enable chunking
        # stride=512,        # Overlap between chunks (optional)
        padding="max_length"
    )
    labels = tokenizer(
        examples["translation"],
        truncation=True,
        max_length=1024,
        return_tensors="pt",
        # return_overflowing_tokens=True,  # Enable chunking
        # stride=512,        # Overlap between chunks (optional)
        padding="max_length"
    )
    inputs["labels"] = labels["input_ids"]
    return inputs

# Apply to dataset
tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=datasets["train"].column_names,
)

In [ ]:
tokenized_datasets

In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # For seq2seq models
    r=32,              # Rank of LoRA matrices (smaller = less memory)
    lora_alpha=32,    # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
    lora_dropout=0.05,
    bias="none",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

In [ ]:
# training_args = TrainingArguments(
#     output_dir=config.OUTPUT_DIR,
#     num_train_epochs=config.NUM_EPOCHS,
#     per_device_train_batch_size=config.BATCH_SIZE,
#     per_device_eval_batch_size=config.BATCH_SIZE,
#     gradient_accumulation_steps=4,
#     warmup_steps=1000,
#     learning_rate=config.LEARNING_RATE,
#     weight_decay=0.01,
#     logging_dir=f"{config.OUTPUT_DIR}/logs",
#     logging_steps=50,
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     save_total_limit=2,
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss",
#     greater_is_better=False,
#     fp16=torch.cuda.is_available(),
#     report_to="tensorboard",
#     remove_unused_columns=True,                             # Saves memory; set False if using custom dataset fields
#     dataloader_pin_memory=True,                             # Helpful on some systems
#     gradient_checkpointing=True,
#     dataloader_num_workers=2,                               # Speeds up data loading; adjust as needed
#     optim="adamw_torch",
# )
training_args = Seq2SeqTrainingArguments(
    output_dir="./nllb-200-3.3B",
    num_train_epochs=config.NUM_EPOCHS,
    per_device_train_batch_size=config.BATCH_SIZE,
    per_device_eval_batch_size=config.BATCH_SIZE,
    gradient_accumulation_steps=8,
    warmup_steps=1000,
    learning_rate=config.LEARNING_RATE,
    weight_decay=0.01,
    logging_dir=f"{config.OUTPUT_DIR}/logs",
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=torch.cuda.is_available(),
    report_to="tensorboard",
    remove_unused_columns=False,                             # Saves memory; set False if using custom dataset fields
    dataloader_pin_memory=True,                             # Helpful on some systems
    dataloader_num_workers=2,                               # Speeds up data loading; adjust as needed
    lr_scheduler_type="cosine",  # Better for LoRA than linear
    max_grad_norm=1.0,  # Gradient clipping
    optim="adamw_torch",
)


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

In [ ]:
trainer.train()

In [21]:
modelx = trainer.model.to("cuda")

In [22]:
import json
def translate_text(input_texts):
    inputs = tokenizer(
        input_texts, 
        return_tensors="pt",
        truncation=True, 
        max_length=config.MAX_LENGTH,
        padding="max_length"
    ).to("cuda")
    
    # Move to GPU if available
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
        modelx.cuda()
        
    # Generate translation
    with torch.no_grad():
        generated_tokens = modelx.generate(
            **inputs,
            forced_bos_token_id=256175,
            num_beams=10,  # ← This parameter
            early_stopping=True
        )
    
    # Decode the translation
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translation

In [23]:
translate_text(data_val['source'][0])

'อาการบาดเจ็บในสมองทําให้มองไม่ชัดและไม่สติ (เพศชาย อายุ 40 ปี)'

In [24]:
data_val['translation'][1]

'เดือนที่แล้ว ประจำเดือนครั้งแรกมาตอนต้นเดือน ครั้งที่สองประจำเดือนหมดวันที่ 22'

In [25]:
import tqdm
predictions = []
for index, text in tqdm.tqdm(data_val.iterrows()):
    predictions.append(translate_text(text['source']))

predictions

3000it [1:40:34,  2.01s/it]


['อาการบาดเจ็บในสมองทําให้มองไม่ชัดและไม่สติ (เพศชาย อายุ 40 ปี)',
 'เดือนที่แล้วผมมีประจําเดือนครั้งแรกในช่วงต้นเดือน และครั้งที่สองในช่วงปลายเดือน คือวันที่ 22',
 'บางคนมีอาการอ่อนแอ และรู้สึกไม่สบาย จะมีอาการปวดและอาการบวม',
 'ความกดดันทางจิตใจสูง ไม่คิดว่าเกี่ยวข้องกับการกินอาหาร จะทําให้คุณลดน้ําหนักมาก หรือเกี่ยวข้องกับโรคเลือดขาดหรือไม่ หมอซอน',
 'ตอนนี้รู้สึกว่าคนทั้งตัวจะมีภาพลวงตา',
 'การหลั่งเลือดนี้ไม่มีความเกี่ยวข้องกับกระดูกที่แพร่กระจาย เป็นเพราะกลัวว่าเนื้องอกจะบุกรังเลือดและหลั่งเลือด',
 'ถ้าเป็นโรคไข้เหลืองติดเชื้อ หรือเป็นโรคไข้เหลืองในเลือด หรือไข้เหลืองสูงมาก อาจเป็นได้',
 'ในกรณีของฉัน คุณแนะนําให้ฉันกินยาอะไร?',
 'ไม่เป็นไร ตอนนี้ยังมีข้อสงสัยอีกหรือไม่ หากคุณรู้สึกว่าคําตอบมีประโยชน์และสะดวก กรุณาประเมินความพึงพอใจของคุณ และสนับสนุนฉัน ไม่จําเป็นต้องพิจารณาความพึงพอใจของคุณ หลักเกณฑ์ของซอฟต์แวร์นี้เป็นพิเศษ และความพึงพอใจยังจะลงโทษฉันได้อีกด้วย',
 'เด็กอึดอัดมาก แต่ก็ผ่านมา 5 วันแล้ว และทําซ้ําซ้ํา',
 'ฉันเห็นว่าการตรวจนิ้วมือทุกครั้งเสร็จสิ้นแล้ว ฉันอ้วนและกลัว

In [26]:
ref = []
ref = data_val['translation']
ref

0       ตาพร่ามัวและหมดสติเนื่องจากสมองบาดเจ็บ (ชายอาย...
1       เดือนที่แล้ว ประจำเดือนครั้งแรกมาตอนต้นเดือน ค...
2                  บางคนแพ้ ทำแล้วไม่สบาย อึดอัดและปวดบวม
3       ความเครียดเยอะ เบื่ออาหารและน้ำ จะทำให้ผอมลงแล...
4              ความรู้สึกตอนนี้คือเหมือนมีอาการประสาทหลอน
                              ...                        
2995    สวัสดี หินปูนเป็นรอยโรคที่หลงเหลืออยู่หลังจากก...
2996            พี่สาวของฉันบอกให้กิน เธอเป็นสูตินรีแพทย์
2997          วิธีการรักษาโรคประสาทอ่อน? (ชาย อายุ 31 ปี)
2998    หลังการสัมผัสเชื้อมา 3 เดือนสามารถยืนยันได้อย่...
2999    ข้อใดข้อหนึ่งก็สามารถตัดความเสี่ยงของการติดเชื...
Name: translation, Length: 3000, dtype: object

In [27]:
syn = {
    '้ํา' : '้ำ',
    '่ํา' : '่ำ',
    '๊ํา' : '๊ำ',
    '๋ํา' : '๋ำ',
    'ํา' : 'ำ'
}
mem = []
for data in predictions:
    s = data
    for k,v in syn.items(): s = s.replace(k,v)
    mem.append(s)
predictions = mem

In [28]:
from pythainlp.tokenize import word_tokenize
import sacrebleu
# Tokenize predictions and references
tokenized_preds = [" ".join(word_tokenize(pred)) for pred in predictions]
tokenized_refs = [" ".join(word_tokenize(r)) for r in ref]

# Compute BLEU with no tokenization (since we already tokenized)
score = sacrebleu.corpus_bleu(tokenized_preds, [tokenized_refs], tokenize="none")
print(f"BLEU score: {score.score:.2f}")
#10.77 r=8
#10.94 r=32
# r=32 && chunk

BLEU score: 19.33


In [29]:
with open("summit_MT/nllb-200-3.3B(fine-tune)(val).txt", "w", encoding="utf-8") as f:
    for item in predictions:
        f.write(f"{item}\n")

In [ ]:
answer = []
for text in tqdm.tqdm(data_test['source']):
    answer.append(translate_text(text))

 84%|████████▍ | 1690/2000 [55:49<10:13,  1.98s/it] 

In [ ]:
syn = {
    '้ํา' : '้ำ',
    '่ํา' : '่ำ',
    '๊ํา' : '๊ำ',
    '๋ํา' : '๋ำ',
    'ํา' : 'ำ'
}
mem = []
for data in answer:
    s = data
    for k,v in syn.items(): s = s.replace(k,v)
    idx = i
    mem.append(s)
answer = mem

In [ ]:
with open("summit_MT/nllb-200-3.3B(fine-tune-max-length=1024).txt", "w", encoding="utf-8") as f:
    for item in answer:
        f.write(f"{item}\n")